In [288]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as sts
from sklearn.metrics import mean_squared_error, mean_absolute_error
import itertools
import random
from tqdm.notebook import tqdm
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import xgboost
warnings.filterwarnings("ignore")

plt.style.use("seaborn")
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [419]:
df = pd.read_csv('/Users/cernenkostanislav/Desktop/Анализ на питон/жиесть.csv')

In [420]:
#некоторые признаки, которые должны быть списками, превратились в строки, исправим
for c in ['grape', 'features', 'taste', 'dishes', 'description']:
    df[c][df[c].isnull() == False] = df[c][df[c].isnull() == False].apply(eval)

# Machine Learning

Данные собраны и обработаны, визуализация сделана, гипотезы проверены, самое время попробовать сделать модель, которая будет определять стоимость вина по его характеристикам.

In [422]:
#разделим на обучающую и тестовую выборки
initial_drop = ['Unnamed: 0', 'name', 'w_id', 'href', 'discount_size', 'discount_price', 'base_price',
               'SW_rating', 'VIVINO_rating', 'WS_rating', 'RVF_rating',
               'JS_rating', 'RP_rating', 'AM_rating', 'GR_rating', 'AIS_rating',
               'PENIN_rating', 'ST_rating', 'JR_rating','description', 'otklonenue']

from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(df.drop(labels = initial_drop ,axis = 1), df['base_price'], test_size=0.2, random_state=3)
X_tr = X_tr.replace(np.nan, None)
X_te = X_te.replace(np.nan, None)

#самостоятельно закодируем с помощью OneHot кодирования признаки, так как они содержат списки
columns_lists = [ 'grape', 'features',
        'taste', 'dishes']
columns_str = ['country', 'color', 'sweetness', 'region','storage_potential', 
               'appellation', 'category', 'decantation','aging_in_container', 'manufacturer']
for c in columns_lists:
    flat_listik = list(itertools.chain(*list(X_tr[c][~X_tr[c].isnull()])))
    for i in list(set(flat_listik)):
        X_tr[i] = in_list_vec(X_tr[c], i)

for c in columns_str:
    flat_listik = X_tr[c][~X_tr[c].isnull()].unique()
    for i in flat_listik:
        X_tr[i] = in_list_vec(X_tr[c], i)

#сделаем transform для тестовой выборки
for c in columns_lists:
    flat_listik = list(itertools.chain(*list(X_tr[c][~X_tr[c].isnull()])))
    for i in list(set(flat_listik)):
        X_te[i] = in_list_vec(X_te[c], i)

for c in columns_str:
    flat_listik = X_tr[c][~X_tr[c].isnull()].unique()
    for i in flat_listik:
        X_te[i] = in_list_vec(X_te[c], i)

In [423]:
#Удалим лишние столбцы
X_tr = X_tr.drop(labels =[ 'grape', 'manufacturer', 'features',
        'taste', 'dishes', 'country', 'color', 'sweetness', 'region','storage_potential', 
               'appellation', 'category', 'decantation','aging_in_container'], axis = 1)
 
X_te = X_te.drop(labels =[ 'grape', 'manufacturer', 'features',
        'taste', 'dishes', 'country', 'color', 'sweetness', 'region','storage_potential', 
               'appellation', 'category', 'decantation','aging_in_container'], axis = 1)

#Заменим None на числа, для рэндом фореста норм
for c in ['sugar', 'acidity', 'aromaticity', 'tannins', 'body', 'client_rating', 'critic_rating']:
    X_tr[c].fillna(value=-999, inplace=True)
    X_te[c].fillna(value=-999, inplace=True)

### Рэндом форест
Мы будем решать задачу регрессии. Так как у нас очень много категориальных признаков, воспользуемся деревьями.

In [377]:
#подберем гиперпараметры
max_depths = range(20, 30)
min_samples_splits = range(2, 10)
results = []
combinations = list(itertools.product(max_depths, min_samples_splits))
for i in tqdm(combinations):
    reg = RandomForestRegressor(n_estimators=100, max_depth=i[0], min_samples_split=i[1])
    scores = cross_val_score(reg, X_tr, y_tr, cv=3, scoring='neg_mean_absolute_error')
    score = np.mean(scores)
    results.append([i, score])
results = np.array(results)
results[results[:,1].argmax()]

  0%|          | 0/80 [00:00<?, ?it/s]

array([(25, 2), -4440.844098910061], dtype=object)

In [424]:
reg = RandomForestRegressor(n_estimators=100,  max_depth=25, min_samples_split=2)
reg.fit(X_tr, y_tr)
y_pred = reg.predict(X_te)
print('MSE на тесте =', mean_squared_error(y_te, y_pred), 'MAE на тесте =', mean_absolute_error(y_te, y_pred)) 
y_pred = reg.predict(X_tr)
print('MSE на трейне =', mean_squared_error(y_tr, y_pred), 'MAE на трейне =', mean_absolute_error(y_tr, y_pred)) 

MSE на тесте = 65009353.404919446 MAE на тесте = 4073.5277025774476
MSE на трейне = 9288903.88814746 MAE на трейне = 1607.7760784901723


Модель переобучилась( Попробуем увеличить число деревьев и снизить глубину

In [384]:
reg = RandomForestRegressor(n_estimators=1000,  max_depth=5, min_samples_split=2)
reg.fit(X_tr, y_tr)
y_pred = reg.predict(X_te)
print('MSE на тесте =', mean_squared_error(y_te, y_pred), 'MAE на тесте =', mean_absolute_error(y_te, y_pred)) 
y_pred = reg.predict(X_tr)
print('MSE на трейне =', mean_squared_error(y_tr, y_pred), 'MAE на трейне =', mean_absolute_error(y_tr, y_pred)) 

MSE на тесте = 111390398.18109967 MAE на тесте = 6048.275592295816
MSE на трейне = 81936072.69532278 MAE на трейне = 5477.128620081566


In [386]:
reg = RandomForestRegressor(n_estimators=1000,  max_depth=15, min_samples_split=2)
reg.fit(X_tr, y_tr)
y_pred = reg.predict(X_te)
print('MSE на тесте =', mean_squared_error(y_te, y_pred), 'MAE на тесте =', mean_absolute_error(y_te, y_pred)) 
y_pred = reg.predict(X_tr)
print('MSE на трейне =', mean_squared_error(y_tr, y_pred), 'MAE на трейне =', mean_absolute_error(y_tr, y_pred)) 

MSE на тесте = 66275573.27677627 MAE на тесте = 4291.862017701811
MSE на трейне = 13901818.550446182 MAE на трейне = 2246.3642531339588


Попробуем увеличить число деревьев по сравнению с первым случаем и сделать их более глубокими, это поможет снизить смещение и разброс.

In [388]:
max_depths = range(25, 35)
results = []
for i in tqdm(max_depths):
    reg = RandomForestRegressor(n_estimators=500, max_depth=i, min_samples_split=2)
    scores = cross_val_score(reg, X_tr, y_tr, cv=3, scoring='neg_mean_absolute_error')
    score = np.mean(scores)
    results.append([i, score])
results = np.array(results)
results[results[:,1].argmax()]

  0%|          | 0/10 [00:00<?, ?it/s]

array([   33.        , -4436.97466336])

In [390]:
reg = RandomForestRegressor(n_estimators=500,  max_depth=33, min_samples_split=2)
reg.fit(X_tr, y_tr)
y_pred = reg.predict(X_te)
print('MSE на тесте =', mean_squared_error(y_te, y_pred), 'MAE на тесте =', mean_absolute_error(y_te, y_pred)) 
y_pred = reg.predict(X_tr)
print('MSE на трейне =', mean_squared_error(y_tr, y_pred), 'MAE на трейне =', mean_absolute_error(y_tr, y_pred)) 

MSE на тесте = 63927448.25410226 MAE на тесте = 4015.4795512982264
MSE на трейне = 9176755.238826793 MAE на трейне = 1514.1605717130103


Еще разок поподбираем

In [397]:
max_depths = range(5, 30)
results = []
for i in tqdm(max_depths):
    reg = RandomForestRegressor(n_estimators=100, max_depth=i, min_samples_split=2)
    scores = cross_val_score(reg, X_tr, y_tr, cv=3, scoring='neg_mean_absolute_error')
    score = np.mean(scores)
    results.append([i, score])
results = np.array(results)
results[results[:,1].argmax()]

  0%|          | 0/25 [00:00<?, ?it/s]

array([   22.        , -4463.46841803])

In [398]:
reg = RandomForestRegressor(n_estimators=100,  max_depth=22, min_samples_split=2)
reg.fit(X_tr, y_tr)
y_pred = reg.predict(X_te)
print('MSE на тесте =', mean_squared_error(y_te, y_pred), 'MAE на тесте =', mean_absolute_error(y_te, y_pred)) 
y_pred = reg.predict(X_tr)
print('MSE на трейне =', mean_squared_error(y_tr, y_pred), 'MAE на трейне =', mean_absolute_error(y_tr, y_pred)) 

MSE на тесте = 66008123.694744974 MAE на тесте = 4122.059537808929
MSE на трейне = 10164188.883606294 MAE на трейне = 1716.250618427761


Сделаем еще один гридсерч, изменив max_depths

In [401]:
max_depths = range(5, 10)
min_samples_splits = range(2, 10)
results = []
combinations = list(itertools.product(max_depths, min_samples_splits))
for i in tqdm(combinations):
    reg = RandomForestRegressor(n_estimators=100, max_depth=i[0], min_samples_split=i[1])
    scores = cross_val_score(reg, X_tr, y_tr, cv=3, scoring='neg_mean_absolute_error')
    score = np.mean(scores)
    results.append([i, score])
results = np.array(results)
results[results[:,1].argmax()]

  0%|          | 0/40 [00:00<?, ?it/s]

array([(9, 6), -5078.692432578854], dtype=object)

In [402]:
reg = RandomForestRegressor(n_estimators=100,  max_depth=9, min_samples_split=6)
reg.fit(X_tr, y_tr)
y_pred = reg.predict(X_te)
print('MSE на тесте =', mean_squared_error(y_te, y_pred), 'MAE на тесте =', mean_absolute_error(y_te, y_pred)) 
y_pred = reg.predict(X_tr)
print('MSE на трейне =', mean_squared_error(y_tr, y_pred), 'MAE на трейне =', mean_absolute_error(y_tr, y_pred)) 

MSE на тесте = 82213044.66984908 MAE на тесте = 5010.166008383767
MSE на трейне = 38180084.998090245 MAE на трейне = 3742.682603400672


Как и ожидалось, качество упало, мы считаем, что это происходит из-за того, что признаков очень много и нужны глубокие деревья, чтобы снизить смещение. 

Качество на трейне растет вместе с качеством на тесте. Значит полученная разница в трейне и тесте говорит о структуре данных, а не о переобученности модели.

__Вывод:__ Лучшие параметры: n_estimators=100,  max_depth=25, min_samples_split=2 Качество рэндом фореста не очень радует, поэтому используем бустинг, может быть он поможет улучшить точность предсказаний. Обучим модель на примерных гиперпарамтерах, после чего попробуем их поменять, чтобы снизить переобучение и уменьшить ошибку на тесте. Мы не будем выполнять GridSearch, так как модель обучается слишком долго, выберем гиперпараметры руками.

In [392]:
model = xgboost.XGBRegressor(n_estimators=1000, booster='gbtree',max_depth=7, subsample=0.5, 
                             learning_rate =0.05)
model.fit(X_tr, y_tr)
y_pred = model.predict(X_te)
print('max_depth=10, subsample=0.5, learning_rate =0.05')
print('MSE тест =', mean_squared_error(y_te, y_pred), 'MAE тест =', mean_absolute_error(y_te, y_pred)) 
y_pred = model.predict(X_tr)
print('MSE трейн =', mean_squared_error(y_tr, y_pred), 'MAE трейн =', mean_absolute_error(y_tr, y_pred)) 

max_depth=10, subsample=0.5, learning_rate =0.05
MSE тест = 43054896.49509551 MAE тест = 3385.1457902129864
MSE трейн = 2775207.431731623 MAE трейн = 1202.1979857990855


Возможно модель переобучилась, уменьшим число деревьев.

In [399]:
model = xgboost.XGBRegressor(n_estimators=500, booster='gbtree',max_depth=7, subsample=0.5, 
                             learning_rate =0.05)
model.fit(X_tr, y_tr)
y_pred = model.predict(X_te)
print('max_depth=10, subsample=0.5, learning_rate =0.05')
print('MSE тест =', mean_squared_error(y_te, y_pred), 'MAE тест =', mean_absolute_error(y_te, y_pred)) 
y_pred = model.predict(X_tr)
print('MSE трейн =', mean_squared_error(y_tr, y_pred), 'MAE трейн =', mean_absolute_error(y_tr, y_pred)) 

max_depth=10, subsample=0.5, learning_rate =0.05
MSE тест = 44871035.150206365 MAE тест = 3581.5222407351584
MSE трейн = 6180447.151665938 MAE трейн = 1717.9855331035076


Стало хуже. Попробуем увеличить)

In [400]:
model = xgboost.XGBRegressor(n_estimators=2000, booster='gbtree',max_depth=7, subsample=0.5, 
                             learning_rate =0.05)
model.fit(X_tr, y_tr)
y_pred = model.predict(X_te)
print('max_depth=10, subsample=0.5, learning_rate =0.05')
print('MSE тест =', mean_squared_error(y_te, y_pred), 'MAE тест =', mean_absolute_error(y_te, y_pred)) 
y_pred = model.predict(X_tr)
print('MSE трейн =', mean_squared_error(y_tr, y_pred), 'MAE трейн =', mean_absolute_error(y_tr, y_pred)) 

max_depth=10, subsample=0.5, learning_rate =0.05
MSE тест = 41833504.998495676 MAE тест = 3231.246299348542
MSE трейн = 825714.3177981823 MAE трейн = 680.5448405721326


Еще один вариант для гиперпараметров.

In [403]:
model = xgboost.XGBRegressor(n_estimators=1500, booster='gbtree',max_depth=7, subsample=0.5, 
                             learning_rate =0.025)
model.fit(X_tr, y_tr)
y_pred = model.predict(X_te)
print('max_depth=10, subsample=0.5, learning_rate =0.05')
print('MSE тест =', mean_squared_error(y_te, y_pred), 'MAE тест =', mean_absolute_error(y_te, y_pred)) 
y_pred = model.predict(X_tr)
print('MSE трейн =', mean_squared_error(y_tr, y_pred), 'MAE трейн =', mean_absolute_error(y_tr, y_pred)) 

max_depth=10, subsample=0.5, learning_rate =0.05
MSE тест = 44199564.393259205 MAE тест = 3507.071264181125
MSE трейн = 3755603.3777808007 MAE трейн = 1378.1331639979453


Мы считаем, что качество может портиться из-за того, что у нас много выбросов. Уберем выбросы, обучим модель.

In [425]:
#Уберем бутылки, которые дороже 90% квантиля
df_clean = df[df['base_price'] < np.quantile(df['base_price'], 0.9)]

In [426]:
#Подготовим новые данные
#разделим на обучающую и тестовую выборки
initial_drop = ['Unnamed: 0', 'name', 'w_id', 'href', 'discount_size', 'discount_price', 'base_price',
               'SW_rating', 'VIVINO_rating', 'WS_rating', 'RVF_rating',
               'JS_rating', 'RP_rating', 'AM_rating', 'GR_rating', 'AIS_rating',
               'PENIN_rating', 'ST_rating', 'JR_rating','description', 'otklonenue']

from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(df_clean.drop(labels = initial_drop ,axis = 1), df_clean['base_price'], test_size=0.2, random_state=3)
X_tr = X_tr.replace(np.nan, None)
X_te = X_te.replace(np.nan, None)

#самостоятельно закодируем с помощью OneHot кодирования признаки, так как они содержат списки
columns_lists = [ 'grape', 'features',
        'taste', 'dishes']
columns_str = ['country', 'color', 'sweetness', 'region','storage_potential', 
               'appellation', 'category', 'decantation','aging_in_container', 'manufacturer']
for c in columns_lists:
    flat_listik = list(itertools.chain(*list(X_tr[c][~X_tr[c].isnull()])))
    for i in list(set(flat_listik)):
        X_tr[i] = in_list_vec(X_tr[c], i)

for c in columns_str:
    flat_listik = X_tr[c][~X_tr[c].isnull()].unique()
    for i in flat_listik:
        X_tr[i] = in_list_vec(X_tr[c], i)

#сделаем transform для тестовой выборки
for c in columns_lists:
    flat_listik = list(itertools.chain(*list(X_tr[c][~X_tr[c].isnull()])))
    for i in list(set(flat_listik)):
        X_te[i] = in_list_vec(X_te[c], i)

for c in columns_str:
    flat_listik = X_tr[c][~X_tr[c].isnull()].unique()
    for i in flat_listik:
        X_te[i] = in_list_vec(X_te[c], i)

#Удалим лишние столбцы
X_tr = X_tr.drop(labels =[ 'grape', 'manufacturer', 'features',
        'taste', 'dishes', 'country', 'color', 'sweetness', 'region','storage_potential', 
               'appellation', 'category', 'decantation','aging_in_container'], axis = 1)
 
X_te = X_te.drop(labels =[ 'grape', 'manufacturer', 'features',
        'taste', 'dishes', 'country', 'color', 'sweetness', 'region','storage_potential', 
               'appellation', 'category', 'decantation','aging_in_container'], axis = 1)

#Заменим None на числа, для рэндом фореста норм
for c in ['sugar', 'acidity', 'aromaticity', 'tannins', 'body', 'client_rating', 'critic_rating']:
    X_tr[c].fillna(value=-999, inplace=True)
    X_te[c].fillna(value=-999, inplace=True)

In [427]:
#Затестим бустинг
model = xgboost.XGBRegressor(n_estimators=1000, booster='gbtree',max_depth=7, subsample=0.5, 
                             learning_rate =0.05)
model.fit(X_tr, y_tr)
y_pred = model.predict(X_te)
print('max_depth=10, subsample=0.5, learning_rate =0.05')
print('MSE тест =', mean_squared_error(y_te, y_pred), 'MAE тест =', mean_absolute_error(y_te, y_pred)) 
y_pred = model.predict(X_tr)
print('MSE трейн =', mean_squared_error(y_tr, y_pred), 'MAE трейн =', mean_absolute_error(y_tr, y_pred)) 

max_depth=10, subsample=0.5, learning_rate =0.05
MSE тест = 9149926.713517064 MAE тест = 1881.1354282206469
MSE трейн = 562183.1626555764 MAE трейн = 562.6490616708431


Ура, качество выросло, видимо выбросы сильно портили модель. Мы понимаем, что в целом МАЕ и MSE зависят от того, какие значения у переменных, и если мы работаем с более дешевыми винами, то МАЕ и MSE будут ниже. Однако, мы считаем, что качественная работа на винах, стоимость которых находится в реалистичном диапазоне, важнее, так как ценообразование элитных вин имеет очень сложный механизм. Попробуем другую модель.

In [428]:
#Затестим рэндом форест
reg = RandomForestRegressor(n_estimators=100,  max_depth=25, min_samples_split=2)
reg.fit(X_tr, y_tr)
y_pred = reg.predict(X_te)
print('MSE на тесте =', mean_squared_error(y_te, y_pred), 'MAE на тесте =', mean_absolute_error(y_te, y_pred)) 
y_pred = reg.predict(X_tr)
print('MSE на трейне =', mean_squared_error(y_tr, y_pred), 'MAE на трейне =', mean_absolute_error(y_tr, y_pred)) 

MSE на тесте = 12537394.657004645 MAE на тесте = 2121.2163988730717
MSE на трейне = 2172964.8156777117 MAE на трейне = 906.5958087944156


__Вывод:__ в результате подбора гиперпараметров, нам не удалось сделать достаточно точную модель. По нашему мнению есть несколько причин, которые могут объяснять качество модели.

__Сложность в кодировании:__ некоторые признаки представляют из себя списки, поэтому нам пришлось применить OneHot кодирование, в результате в тесте могут встречаться признаки, которых нет в трейне, поэтому они не отображаются при кодировке в тестовой выборке, а значит не учитываются при предсказании.

__Структура данных:__ в датасете встречаются значения признаков, которые принадлежат лишь небольшой доле бутылок, они могут не попасть в трейн и не учитываться при обучении.

__Наличие других признаков:__ мы предполагаем, что существуют другие признаки, которые лучше коррелируют со стоимостью, возможно для этого стоит использовать другой источник данных.